# 1 Preparations

### 1.1 Set Options

In [ ]:
### Set options here
#Structural options
market = 'FullYear' #Choose from ['Balancing', 'DayAhead', 'FullYear', 'Investment']
SCENARIO = 'all' #Add scenario to read file name
YEAR = 'full' #Add year to read file name (e.g. '2025', '2035', 'full')
SUBSET = 'full'
year = 2045 #Year to be displayed
LINES = 'CongestionFlow' #Choose from: ['Capacity', 'Flow', 'CongestionFlow']. For 'CongestionFlow', exo_end automatically switches to 'Total'.
exo_end = 'Total' # Choose from ['Endogenous', 'Exogenous', 'Total']. For 'CongestionFlow', exo_end automatically switches to 'Total'.
S = 'S38' #Season 
T = 'T088' #Hour

#Visual options
label_min = 0 #Minimum transmission capacity (GW) shown on map in text
font_line = 10 #Font size of transmission line labels
font_hub = 10 #Font size of hub labels
font_region = 12 #Font size of region labels
line_decimals = 1 #Number of decimals shown for line capacities
line_width_constant = 5 #Constant related to thickness of lines: the higher the number, the narrower the lines will be
flowline_breaks = [0, 50, 94.999, 100] #Breaks for different congestion categories
hub_display = True
hub_size = 10
hub_decimals = 0 #Number of decimals shown for hub capacities
background_hubsize = True #Displaying the true size of the hub as a circle on the map.
hub_area = 6.4 #MW / km^2, background hub size on map. 
hub_area_opacity = 0.3 #Opacity of background hub size. 


#Colours
background_colour = 'white'
regions_ext_colour = 'lightgrey'
regions_model_colour = 'grey'
region_text = 'black'
capline_colour = 'orange'
flowline_colour = ['#3D9200', '#feb24c','#960028'] #NB: Change legend colours accordingly, in the cell under "3.6 Add Legend": lines 43-45
line_text = 'black'
hub_colour = 'lightblue'
hub_background_colour = 'lightblue'
hub_text = 'black'

### 1.2 Import Packages

In [ ]:
from pathlib import Path
import sys
import os
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from folium import plugins
from IPython.display import HTML, display
import json
from folium.features import DivIcon #For text labels on hubs
from IPython.display import display, HTML
from csv import reader
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

### 1.3 Import Files

In [ ]:
#Set file names
project_dir = Path('./input')
map_name = 'Transmission_' + LINES + '_' + str(year) + '_Map.html'
flow_file = 'FlowElectricityHourly_'+ SCENARIO + '_' + YEAR + '_' + SUBSET + '.csv'
transcap_file = 'CapacityPowerTransmission_'+ SCENARIO + '_' + YEAR + '_'+ SUBSET + '.csv'
generation_file = 'CapacityGeneration_'+ SCENARIO + '_' + YEAR + '_' + SUBSET + '.csv'
prod_file = 'ProductionHourly_'+ SCENARIO + '_' + YEAR + '_' + SUBSET + '.csv'


#Load coordinates files 
df_unique = pd.read_csv(project_dir/'transmission_files/coordinates_RRR.csv')
df_region = df_unique.loc[df_unique['Type'] == 'region', ]
df_bypass = pd.read_csv(project_dir/'transmission_files/bypass_lines.csv') # coordinates of 'hooks' in indirect lines, to avoid going trespassing third regions


#Transmission capacity data
df_capacity = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(transcap_file), sep = ',', quotechar = '"') 
#Transmission flow data
df_flow = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(flow_file), sep = ',', quotechar = '"')
#Generation capacity data
df_capgen = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(generation_file), sep = ',', quotechar = '"') 
#Hub production data
hub_windgen = (pd.read_csv(project_dir/'transmission_files/hub_names.csv', sep = ',', quotechar = '"').hub_name) 
df_hubprod = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(prod_file), sep = ',', quotechar = '"') 
df_hubprod = df_hubprod.loc[(df_hubprod['G'].isin(hub_windgen)) & (df_hubprod['TECH_TYPE'] == 'WIND-OFF') & \
                            (df_hubprod['Y']==year) & (df_hubprod['SSS'] == S) & (df_hubprod['TTT']==T), ]

#Define names of geojson and shapefile layers
r_in = list(df_unique.loc[(df_unique['Display'] == 1) & (df_unique['Type'] == 'region'), 'RRR'])
r_out = list(df_unique.loc[(df_unique['Display'] == 0) & (df_unique['Type'] == 'region'), 'RRR'])

layers_in = {region: '' for region in r_in}
layers_out = {region: '' for region in r_out}

#Create dictionaries with layer names for each region; if both a shapefile and geojson file are available for one region, the geojson file is used. 
for region in r_in:
    layers_in[region] = glob.glob(f'{project_dir}/transmission_files/geojson_files/'+ region + '.geojson')
    if bool(layers_in[region]) == False:
        layers_in[region] = glob.glob(f'{project_dir}/transmission_files/shapefiles/'+ region + '.shp')
for region in r_out:
    layers_out[region] = glob.glob(f'{project_dir}/transmission_files/geojson_files/'+ region + '.geojson')
    if bool(layers_out[region]) == False:
        layers_out[region] = glob.glob(f'{project_dir}/transmission_files/shapefiles/'+ region + '.shp')

for region in layers_in:
    layers_in[region] = str(layers_in[region])[2:-2] #Remove brackets from file names
for region in layers_out:
    layers_out[region] = str(layers_out[region])[2:-2] #Remove brackets from file names


In [ ]:
for region in layers_out:
    if layers_out[region][-4:] == '.shp':
        gpd.read_file(layers_out[region]).to_file(f'{project_dir}/transmission_files/geojson_files/'+ region + '.geojson', driver='GeoJSON')
        layers_out[region] = layers_out[region].replace('shapefiles', 'geojson_files').replace('.shp', '.geojson')

# 2 Processing of dataframes

### 2.1 Replace "EPS" with 0

In [ ]:
#Replace possible "Eps" with 0
df_capacity.Val=df_capacity.Val.replace('Eps', 0)
df_capacity.Val=pd.to_numeric(df_capacity.Val)
df_flow.Val=df_flow.Val.replace('Eps', 0)
df_flow.Val=pd.to_numeric(df_flow.Val)
df_capgen.Val=df_capgen.Val.replace('Eps', 0)
df_capgen.Val=pd.to_numeric(df_capgen.Val)
df_hubprod.Val=df_hubprod.Val.replace('Eps', 0)
df_hubprod.Val=pd.to_numeric(df_hubprod.Val)

### 2.2 Add Coordinates + Select Time + Convert Units

In [ ]:
#Flows
if LINES == 'Flow' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Capacity'
    #Keep only data from moment of interest
    df_flow = df_flow.loc[df_flow['Y'] == year] 
    df_flow = df_flow.loc[df_flow['SSS'] == S,]
    df_flow = df_flow.loc[df_flow['TTT'] == T, ]
    for i,row in df_flow.iterrows():
        for j in range(0,len(df_unique)):
            if df_flow.loc[i,'IRRRE'] == df_unique.loc[j, 'RRR']:
                df_flow.loc[i,'LatExp'] = df_unique.loc[j, 'Lat']
                df_flow.loc[i,'LonExp'] = df_unique.loc[j, 'Lon']
            if df_flow.loc[i,'IRRRI'] == df_unique.loc[j, 'RRR']:
                df_flow.loc[i,'LatImp'] = df_unique.loc[j, 'Lat']
                df_flow.loc[i,'LonImp'] = df_unique.loc[j, 'Lon']

    #Convert flow from MWh to GWh
    df_flow['Val'] = df_flow['Val'] / 1000
    df_flow = df_flow.reset_index(drop = True)
    if len(df_flow) == 0:
        print("Error: Timestep not in data; check year, S and T.")
        sys.exit()

### 2.3 Group hub data

In [ ]:
#Generation Capacities
df_capgen = df_capgen.merge(df_unique, on = 'RRR', how = 'left', left_index = True).reset_index(drop = True) #Add coordinates of each region
df_capgen = df_capgen.loc[df_capgen['Y'] == year] #Keep only data from year of interest
df_hubcap = df_capgen.loc[df_capgen['G'].isin(hub_windgen),] #Keep only hub data 
df_hubcap_agg = pd.DataFrame(df_hubcap.groupby(['Y', 'C', 'RRR', 'Lat', 'Lon'])['Val'].sum().reset_index()) #Sum all capacities (of different wind turbines) at each location
df_hubcap_agg['Radius'] = np.sqrt(df_hubcap_agg['Val'] * 1000 / hub_area / np.pi) # Create column of hub radius (in kilometres)

#Merge all relevant hub info into one dataframe
df_hubprod = pd.DataFrame(df_hubprod.groupby(['Y', 'C', 'RRR'])['Val'].sum().reset_index()) #Sum all production (of different wind turbines) at each location
df_hubprod.Val = df_hubprod.Val/1000
df_hubprod.rename(columns = {'Val': 'prod_GWh'}, inplace = True)
df_hub = pd.merge(df_hubcap_agg, df_hubprod[['RRR', 'prod_GWh']], on = 'RRR', how = 'left', left_index = True).reset_index(drop = True) 

### 2.4 Prepare capacity dataframe

In [ ]:
#Transmission Capacities
if LINES == 'Capacity' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Flow'
    df_capacity = df_capacity.loc[df_capacity['Y'] == year, ].reset_index(drop = True) #Keep only data from year of interest

    if exo_end == 'Total' or LINES == 'CongestionFlow':
        col_keep = list(np.delete(np.array(df_capacity.columns),np.where((df_capacity.columns == 'VARIABLE_CATEGORY') | \
                                    (df_capacity.columns == 'Val')) )) #Create list with all columns except 'Variable_Category' and 'Val'
        df_capacity = pd.DataFrame(df_capacity.groupby(col_keep)['Val'].sum().reset_index() )#Sum exogenous and endogenous capacity for each region
    if exo_end == 'Endogenous' and LINES != 'CongestionFlow':
        df_capacity = df_capacity.loc[df_capacity['VARIABLE_CATEGORY'] == 'ENDOGENOUS', ]
    if exo_end == 'Exogenous' and LINES != 'CongestionFlow':
        df_capacity = df_capacity.loc[df_capacity['VARIABLE_CATEGORY'] == 'EXOGENOUS', ]

    for i,row in df_capacity.iterrows():
        for j in range(0,len(df_unique)):
            if df_capacity.loc[i,'IRRRE'] == df_unique.loc[j, 'RRR']:
                df_capacity.loc[i,'LatExp'] = df_unique.loc[j, 'Lat']
                df_capacity.loc[i,'LonExp'] = df_unique.loc[j, 'Lon']
            if df_capacity.loc[i,'IRRRI'] == df_unique.loc[j, 'RRR']:
                df_capacity.loc[i,'LatImp'] = df_unique.loc[j, 'Lat']
                df_capacity.loc[i,'LonImp'] = df_unique.loc[j, 'Lon']
    if len(df_capacity) == 0:
        print("Error: No capacity found. Check year and exo_end.")
        sys.exit()

### 2.5 Add bypass coordinates for indirect lines

In [ ]:
if LINES == 'Capacity':
    df_bypass = pd.merge(df_bypass, df_capacity[['Y', 'C', 'IRRRE', 'IRRRI', 'UNITS', 'Val']], on = ['IRRRE', 'IRRRI'], how = 'left')
    #Replace existing row by 2 bypass rows
    keys = list(df_bypass.columns.values)[0:2]
    i1 = df_capacity.set_index(keys).index
    i2 = df_bypass.set_index(keys).index
    df_capacity = df_capacity[~i1.isin(i2)] #Delete existing rows that need bypass
    df_capacity = df_capacity.append(df_bypass, ignore_index = True, sort = True) #Append bypass rows
    
if LINES == 'Flow' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Capacity'
    df_bypass = pd.merge(df_bypass, df_flow[['Y', 'C', 'IRRRE', 'IRRRI', 'SSS', 'TTT', 'UNITS', 'Val']], on = ['IRRRE', 'IRRRI'], how = 'left').dropna()
    #Replace existing row by 2 bypass rows
    keys = list(df_bypass.columns.values)[0:2]
    i1 = df_flow.set_index(keys).index
    i2 = df_bypass.set_index(keys).index
    df_flow = df_flow[~i1.isin(i2)]#Delete existing rows that need bypass
    df_flow = df_flow.append(df_bypass, ignore_index = True, sort = True)#Append bypass rows

### 2.6 Calculate Congestion

In [ ]:
if LINES == 'CongestionFlow': #Skip this cell in case LINES != 'CongestionFlow'
    df_flow = pd.merge(df_flow, df_capacity[['Y', 'C', 'IRRRE', 'IRRRI', 'Val']], on = ['Y', 'C', 'IRRRE', 'IRRRI'], how = 'left')
    df_flow.rename(columns={'Val_x': 'Val', 'Val_y' : 'Capacity'}, inplace = True)
    df_flow['Congestion'] = df_flow['Val'] / df_flow['Capacity'] * 100

    #Create colour codes for congestion of lines
    df_flow['Colour'] = pd.cut(df_flow['Congestion'], bins = flowline_breaks, labels = flowline_colour )

### 2.7 One direction capacity  lines

In [ ]:
#When capacity is not the same in both directions, display one:
for i,row in df_capacity.iterrows():
    for k,row in df_capacity.iterrows():
        if (df_capacity.loc[k,'IRRRE'] == df_capacity.loc[i,'IRRRI']) & (df_capacity.loc[k,'IRRRI'] == df_capacity.loc[i,'IRRRE']) & (df_capacity.loc[k,'Val'] != df_capacity.loc[i,'Val']):
            df_capacity.loc[i,'Val'] = df_capacity.loc[k,'Val']

###  2.8 Define line centers

In [ ]:
#Define centre of each transmission line
if LINES == 'Flow' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Capacity'
    df_flow['LatMid'] = (df_flow['LatImp'] + df_flow['LatExp']) /2
    df_flow['LonMid'] = (df_flow['LonImp'] + df_flow['LonExp']) /2
if LINES == 'Capacity' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Flow'
    df_capacity['LatMid'] = (df_capacity['LatImp'] + df_capacity['LatExp']) /2
    df_capacity['LonMid'] = (df_capacity['LonImp'] + df_capacity['LonExp']) /2

# 3 Create Map Features

### 3.1 Create map

In [ ]:
#Create map 
map_center = [55.220228, 10.419778]
m = folium.Map(location= map_center, zoom_start=5, tiles='')
#Add background layers (sea, regions in model, countries outside of model)
folium.Polygon(locations = [[-90,-180], [90,-180], [90,180], [-90,180]], color = background_colour, fill_color = background_colour, opacity = 1, fill_opacity = 1 ).add_to(m) #Background
#folium.GeoJson(data = DE_shp)

for region in layers_in: 
    folium.GeoJson(data = layers_in[region], name = 'regions_in', \
               style_function = lambda x:{'fillColor': regions_model_colour, 'fillOpacity': 0.5, 'color': regions_model_colour, 'weight':1}).add_to(m) #Regions within model
for region in layers_out: 
    folium.GeoJson(data = layers_out[region], name = 'regions_out', \
                   style_function = lambda x:{'fillColor': regions_ext_colour, 'fillOpacity': 0.5, 'color': regions_ext_colour, 'weight':1}).add_to(m) #Neighbouring countries


### 3.2 Create background hub size

In [ ]:
if hub_display == True:
    if background_hubsize == True:
        if LINES == 'Capacity': 
            for i,row in df_hub.iterrows():
                folium.Circle(
                  location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                  popup=df_hub.loc[i,'RRR'],
                  radius = df_hub.loc[i,'Radius']*1000,
                  color = hub_background_colour,
                  opacity = 0,
                  fill=True,
                  fill_color = hub_background_colour,
                  fill_opacity = hub_area_opacity
               ).add_to(m)
        if LINES == 'Flow' or LINES == 'CongestionFlow':
            for i,row in df_hub.iterrows():
                folium.Circle(
                  location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                  popup=df_hub.loc[i,'RRR'],
                  radius = df_hub.loc[i,'Radius']*1000,
                  color = hub_background_colour,
                  opacity = 0,
                  fill=True,
                  fill_color = hub_background_colour,
                  fill_opacity = hub_area_opacity
               ).add_to(m)

### 3.3 Add lines

In [ ]:
#Add capacity lines
if LINES == 'Capacity':
    for i,row in df_capacity.iterrows():
        folium.PolyLine(([df_capacity.loc[i,'LatExp'], df_capacity.loc[i,'LonExp']], \
                             [df_capacity.loc[i,'LatImp'],df_capacity.loc[i,'LonImp']]), \
                            color=capline_colour, line_cap = 'butt', weight=df_capacity.loc[i,'Val']/line_width_constant, opacity=1).add_to(m)  
        if df_capacity.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_capacity.loc[i,'LatMid'], df_capacity.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            df_capacity.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_capacity.loc[i,'LatMid'], df_capacity.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            round(df_capacity.loc[i,'Val'],line_decimals)))).add_to(m)
#Add flows (single colour)                
if LINES == 'Flow':
    attr = {'font-weight': 'bold', 'font-size': '24'}
    for i,row in df_flow.iterrows():
        flow = folium.PolyLine(([df_flow.loc[i,'LatExp'], df_flow.loc[i,'LonExp']], \
                             [df_flow.loc[i,'LatImp'],df_flow.loc[i,'LonImp']]), \
                            color=capline_colour, line_cap = 'butt', weight=df_flow.loc[i,'Val']/line_width_constant, opacity=1).add_to(m)   
        plugins.PolyLineTextPath(flow, '\u2192', repeat=False ,center = True, offset=6, orientation = -90, \
                                 attributes=attr).add_to(m)  #Arrow
        if df_flow.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            df_flow.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            round(df_flow.loc[i,'Val'],line_decimals)))).add_to(m)    
#Add flows (colour based on congestion)
if LINES == 'CongestionFlow':
    attr = {'font-weight': 'bold', 'font-size': '24'}
    for i,row in df_flow.iterrows():
        flow = folium.PolyLine(([df_flow.loc[i,'LatExp'], df_flow.loc[i,'LonExp']], \
                             [df_flow.loc[i,'LatImp'],df_flow.loc[i,'LonImp']]), \
                            color=df_flow.loc[i,'Colour'], line_cap = 'butt', weight=df_flow.loc[i,'Val']/line_width_constant, opacity=1).add_to(m)   
        plugins.PolyLineTextPath(flow, '\u2192', repeat=False ,center = True, offset=6, orientation = -90, \
                                 attributes=attr).add_to(m)  #Arrow
        if df_flow.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            df_flow.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, \
                            round(df_flow.loc[i,'Val'],line_decimals)))).add_to(m)    

### 3.4 Add region names

In [ ]:
#Add region names
for i,row in df_region.loc[df_region['Display']==1, ].iterrows():
    folium.Marker(location=[df_region.loc[i,'Lat'], df_region.loc[i,'Lon']],
                  icon=DivIcon(
                      icon_size=(150,36), 
                               icon_anchor=(7,7),
     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_region, region_text, df_region.loc[i,'RRR']))).add_to(m)  

### 3.5 Add hubs

In [ ]:
#Add hub capacities as bubbles
if hub_display == True:
    if LINES == 'Capacity': 
        for i,row in df_hub.iterrows():
            folium.CircleMarker(
              location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
              popup=df_hub.loc[i,'RRR'],
              radius = hub_size,
              color= hub_colour,
              opacity = 0,
              fill=True,
              fill_color= hub_colour,
              fill_opacity = 1
           ).add_to(m)

            if hub_decimals == 0:
                folium.Marker(location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                              icon=DivIcon(
                                  icon_size=(150,36), 
                                           icon_anchor=(7,9),
                 html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, df_hub.loc[i,'Val'].round(hub_decimals).astype(int)))).add_to(m)
            else:
                folium.Marker(location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                              icon=DivIcon(
                                  icon_size=(150,36), 
                                           icon_anchor=(7,9),
                 html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, round(df_hub.loc[i,'Val'], hub_decimals)))).add_to(m)    

    if LINES == 'Flow' or LINES == 'CongestionFlow':
        for i,row in df_hub.iterrows():
            folium.CircleMarker(
              location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
              popup=df_hub.loc[i,'RRR'],
              radius = hub_size,
              color= hub_colour,
              opacity = 0,
              fill=True,
              fill_color= hub_colour,
              fill_opacity = 1
           ).add_to(m)

            if hub_decimals == 0:
                folium.Marker(location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                              icon=DivIcon(
                                  icon_size=(150,36), 
                                           icon_anchor=(7,9),
                 html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, df_hub.loc[i,'prod_GWh'].round(hub_decimals).astype(int)))).add_to(m)
            else:
                folium.Marker(location=[df_hub.loc[i,'Lat'], df_hub.loc[i,'Lon']],
                              icon=DivIcon(
                                  icon_size=(150,36), 
                                           icon_anchor=(7,9),
                 html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, round(df_hub.loc[i,'prod_GWh'], hub_decimals)))).add_to(m)    


### 3.6 Add Legend

In [ ]:
if LINES == 'CongestionFlow':
    from branca.element import Template, MacroElement

    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 1);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Congestion rate</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:#960028;opacity:0.7;'></span>Fully congested</li>
        <li><span style='background:#feb24c;opacity:0.7;'></span> 50-95% congested </li>
        <li><span style='background:#3D9200;opacity:0.7;'></span> <50 % congested </li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    m.get_root().add_child(macro)

### 4 Save Output

In [ ]:
# Make Transmission_Map output folder
if not os.path.isdir('output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market):
    os.makedirs('output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market)

In [ ]:
output_dir = 'output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market
m.save(output_dir + '/' +  map_name)

### 5 Display Map

In [ ]:
m